In [3]:
import requests
import time
import hashlib
import hmac
import os
import pandas as pd
from decimal import Decimal

# Test
API_KEY = "d727ada381ac80bb187e04ca361e0e0f6ba5f6fc22d3cbf610c17c32d936657e"
API_SECRET = "2f33bf9f838e705446915b60dfa4440c2303d478a7a726536bd0b264c2b1ec45"
URL = "https://testnet.binancefuture.com"

#BASE_URL = 'https://fapi.binance.com'
BASE_URL = "https://testnet.binancefuture.com"



def create_signature(query_string, secret):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def binance_request(endpoint, params):
    params['timestamp'] = int(time.time() * 1000)
    query_string = '&'.join([f"{key}={value}" for key, value in sorted(params.items())])
    signature = create_signature(query_string, API_SECRET)
    headers = {
        'X-MBX-APIKEY': API_KEY
    }
    response = requests.get(f"{BASE_URL}{endpoint}?{query_string}&signature={signature}", headers=headers)
    return response.json()

def binance_post_request(endpoint, params):
    params['timestamp'] = int(time.time() * 1000)
    query_string = '&'.join([f"{key}={value}" for key, value in sorted(params.items())])
    signature = create_signature(query_string, API_SECRET)
    headers = {
        'X-MBX-APIKEY': API_KEY
    }
    response = requests.post(f"{BASE_URL}{endpoint}?{query_string}&signature={signature}", headers=headers)
    return response.json()

def get_positions():
    return binance_request('/fapi/v2/positionRisk', {})

def get_pnl():
    return binance_request('/fapi/v2/account', {})

def get_price(symbol):
    params = {'symbol': symbol}
    return binance_request('/fapi/v1/ticker/price', params)

def get_ohlc_data(symbol, start_date, end_date, interval):
    base_url = "https://api.binance.com"
    endpoint = "/api/v3/klines"

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": int(start_date.timestamp() * 1000),
        "endTime": int(end_date.timestamp() * 1000),
        "limit": 500
    }

    response = requests.get(base_url + endpoint, params=params)
    data = response.json()

    if response.status_code == 200:
        ohlc_data = []
        for entry in data:
            ohlc_data.append({
                "time": pd.to_datetime(entry[0], unit='ms'),
                "open": float(entry[1]),
                "high": float(entry[2]),
                "low": float(entry[3]),
                "close": float(entry[4])
            })
        return ohlc_data
    else:
        return None


def get_open_orders():
    return binance_request('/fapi/v1/openOrders', {})

def get_filters(symbol):
    response = binance_request('/fapi/v1/exchangeInfo', {})
    symbol_info = next((item for item in response['symbols'] if item['symbol'] == symbol), None)
    if symbol_info:
        filters = {f['filterType']: f for f in symbol_info['filters']}
        return filters
    return None

def get_average_price(symbol):
    response = binance_request('/fapi/v1/avgPrice', {'symbol': symbol})
    return float(response['price'])

def get_open_orders_count(symbol):
    orders = get_open_orders()
    return sum(1 for order in orders if order['symbol'] == symbol)

def get_current_position(symbol):
    positions = get_positions()
    position = next((p for p in positions if p['symbol'] == symbol), None)
    return float(position['positionAmt']) if position else 0.0

def validate_order(symbol, quantity, price=None):
    filters = get_filters(symbol)
    if not filters:
        return {'code': -1, 'msg': 'No filters found for the symbol'}

    # Validate LOT_SIZE
    lot_size = filters.get('LOT_SIZE')
    if lot_size:
        min_qty = float(lot_size['minQty'])
        max_qty = float(lot_size['maxQty'])
        step_size = float(lot_size['stepSize'])
        if quantity < min_qty or quantity > max_qty or (Decimal(str('quantity')) - Decimal(str('min_qty'))) % Decimal(str('step_size')) != 0:
            return {'code': -1, 'msg': f'Invalid quantity: {quantity}. Must be between {min_qty} and {max_qty}, in multiples of {step_size}'}

    # Validate MARKET_LOT_SIZE
    market_lot_size = filters.get('MARKET_LOT_SIZE')
    if market_lot_size:
        min_qty = float(market_lot_size['minQty'])
        max_qty = float(market_lot_size['maxQty'])
        step_size = float(market_lot_size['stepSize'])
        if quantity < min_qty or quantity > max_qty or (Decimal(str('quantity')) - Decimal(str('min_qty'))) % Decimal(str('step_size')) != 0:
            return {'code': -1, 'msg': f'Invalid market order quantity: {quantity}. Must be between {min_qty} and {max_qty}, in multiples of {step_size}'}

    # Validate PRICE_FILTER
    price_filter = filters.get('PRICE_FILTER')
    if price and price_filter:
        min_price = float(price_filter['minPrice'])
        max_price = float(price_filter['maxPrice'])
        tick_size = float(price_filter['tickSize'])
        if price < min_price or price > max_price or (Decimal(str('price')) - Decimal(str('min_price'))) % Decimal(str('tick_size')) != 0:
            return {'code': -1, 'msg': f'Invalid price: {price}. Must be between {min_price} and {max_price}, in multiples of {tick_size}'}

    # Validate MIN_NOTIONAL
    min_notional = filters.get('MIN_NOTIONAL')
    if min_notional and price:
        notional = quantity * price
        min_notional_value = float(min_notional['notional'])
        if notional < min_notional_value:
            return {'code': -1, 'msg': f'Invalid notional value: {notional}. Must be at least {min_notional_value}'}

    # Validate PERCENT_PRICE
    percent_price = filters.get('PERCENT_PRICE')
    if percent_price and price:
        avg_price = get_average_price(symbol)
        multiplier_up = float(percent_price['multiplierUp'])
        multiplier_down = float(percent_price['multiplierDown'])
        if price > avg_price * multiplier_up or price < avg_price * multiplier_down:
            return {'code': -1, 'msg': f'Invalid price: {price}. Must be within {multiplier_down*100}% and {multiplier_up*100}% of the average price {avg_price}'}

    # Validate MAX_NUM_ORDERS
    max_num_orders = filters.get('MAX_NUM_ORDERS')
    if max_num_orders:
        num_orders = get_open_orders_count(symbol)
        max_orders = int(max_num_orders['limit'])
        if num_orders >= max_orders:
            return {'code': -1, 'msg': f'Exceeded maximum number of open orders: {max_orders}'}

    # Validate MAX_NUM_ALGO_ORDERS
    max_num_algo_orders = filters.get('MAX_NUM_ALGO_ORDERS')
    if max_num_algo_orders:
        num_algo_orders = get_open_orders_count(symbol)  # Simplified: you might need a separate count for algo orders
        max_algo_orders = int(max_num_algo_orders['limit'])
        if num_algo_orders >= max_algo_orders:
            return {'code': -1, 'msg': f'Exceeded maximum number of algo orders: {max_algo_orders}'}

    # Validate MAX_POSITION
    max_position = filters.get('MAX_POSITION')
    if max_position:
        current_position = get_current_position(symbol)
        max_pos = float(max_position['maxPosition'])
        if current_position + quantity > max_pos:
            return {'code': -1, 'msg': f'Exceeded maximum position size: {max_pos}'}

    return {'code': 0, 'msg': 'Order is valid'}

def suggest_order_adjustments(symbol, quantity, price=None):
    filters = get_filters(symbol)
    if not filters:
        return {'code': -1, 'msg': 'No filters found for the symbol'}

    suggestions = {}

    # Suggest LOT_SIZE adjustments
    lot_size = filters.get('LOT_SIZE')
    if lot_size:
        step_size = float(lot_size['stepSize'])
        if quantity % step_size != 0:
            suggested_qty = quantity - (quantity % step_size) + step_size
            suggestions['quantity'] = suggested_qty

    # Suggest PRICE_FILTER adjustments
    price_filter = filters.get('PRICE_FILTER')
    if price and price_filter:
        tick_size = float(price_filter['tickSize'])
        if price % tick_size != 0:
            suggested_price = price - (price % tick_size) + tick_size
            suggestions['price'] = suggested_price

    return {'code': 0, 'msg': 'Order suggestions provided', 'suggestions': suggestions}

def place_order(symbol, side, order_type, quantity, price=None, stop_price=None):
    validation = validate_order(symbol, quantity, price)
    if validation['code'] != 0:
        adjustments = suggest_order_adjustments(symbol, quantity, price)
        return {
            'code': validation['code'],
            'msg': validation['msg'],
            'adjustments': adjustments.get('suggestions', {})
        }

    params = {
        'symbol': symbol,
        'side': side,
        'type': order_type,
        'quantity': quantity
    }
    if price:
        params['price'] = price
    if stop_price:
        params['stopPrice'] = stop_price

    return binance_post_request('/fapi/v1/order', params)


In [4]:
from app.place_order import place_order

%load_ext autoreload
%autoreload 2
 

place_order(symbol = "ETHUSDT", side="BUY", order_type="MARKET", quantity=0.1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ETHUSDT


{'orderId': 1384997429,
 'symbol': 'ETHUSDT',
 'status': 'NEW',
 'clientOrderId': 'RYPf719pRs98QCjiO7mEAE',
 'price': '0.00',
 'avgPrice': '0.00',
 'origQty': '0.100',
 'executedQty': '0.000',
 'cumQty': '0.000',
 'cumQuote': '0.00000',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'BUY',
 'positionSide': 'BOTH',
 'stopPrice': '0.00',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'MARKET',
 'priceMatch': 'NONE',
 'selfTradePreventionMode': 'NONE',
 'goodTillDate': 0,
 'updateTime': 1717363030802}

In [ ]:
def get_mark_price(symbol):
    params = {"symbol": symbol}
    return binance_request('/fapi/v1/premiumIndex', params)

get_mark_price("QNTUSDT")["markPrice"]

In [ ]:
get_filters(symbol="QNTUSDT")

In [ ]:

def validate_order(symbol, quantity, side, price=None):
    filters = get_filters(symbol)
    if not filters:
        return {'code': -1, 'msg': 'No filters found for the symbol'}
    
    msg = {}
    sugg = {}
    
    # LOT_SIZE
    lot_size = filters.get('LOT_SIZE')
    if lot_size:
        min_qty = lot_size['minQty']
        max_qty = lot_size['maxQty']
        step_size = lot_size['stepSize']
        
        if quantity < float(min_qty):
            msg["minQty"] = f"Invalid Qty. Must be > {min_qty}"
            sugg["qty"] = float(min_qty)
        if quantity > float(max_qty):
            msg["maxQty"] = f"Invalid Qty. Must be < {max_qty}"
            sugg["qty"] = float(max_qty)
        if (Decimal(str(quantity)) - Decimal(min_qty)) % Decimal(step_size) != 0:
            msg["stepSize"] = f"Invalid stepSize"
            r = (Decimal(str(quantity)) - Decimal(min_qty)) % Decimal(step_size)
            sugg["qty"] = float(Decimal(str(quantity)) - r)
    
    # PRICE_FILTER
    price_filter = filters.get("PRICE_FILTER")
    if price_filter and price:
        min_price = price_filter["minPrice"]
        max_price = price_filter["maxPrice"]
        tick_size = price_filter["tickize"]
        if price < float(min_price):
            msg["minPrice"] = f"Invalid Price. Must be > {min_price}"
        if price > float(max_price):
            msg["maxPrice"] = f"Invalid Price. Must be < {max_price}"
        if (Decimal(str(price)) - Decimal(min_price)) % Decimal(tick_size) != 0:
            msg["tickSize"] = f"Invalid Tick Size"
    
     # PERCENT_PRICE
    percent_price = filters.get('PERCENT_PRICE')
    if percent_price and price:
        multiplierUp = percent_price.get('multiplierUp')
        multiplierDown = percent_price.get('multiplierDown')
        multiplierUPDec = percent_price.get('multiplierDecimal')
        mark_price = get_mark_price(symbol)['markPrice']
        
        if (side == "BUY") and (price > float(mark_price )*float(multiplierUp)):
            msg["percentPrice"] = f"BUY : Invalid"
        if (side == "SELL") and (price < float(mark_price) * float(multiplierDown)):
            msg["percentPrice"] = f"SELL : Invalid"
            
    
    # MARKET_LOT_SIZE
    market_lot_size = filters.get('MARKET_LOT_SIZE')
    if market_lot_size:
        m_min_qty = market_lot_size['minQty']
        m_max_qty = market_lot_size['maxQty']
        m_step_size = market_lot_size['stepSize']
        if quantity < float(m_min_qty):
            msg["MminQty"] = f"Invalid Qty. Must be > {m_min_qty}"
            sugg["Mqty"] = float(m_min_qty)
        if quantity > float(m_max_qty):
            msg["MmaxQty"] = f"Invalid Qty. Must be < {m_max_qty}"
            sugg["Mqty"] = float(m_max_qty)
        if (Decimal(str(quantity)) - Decimal(m_min_qty)) % Decimal(m_step_size) != 0:
            msg["MstepSize"] = f"Invalid stepSize"
            r = (Decimal(str(quantity)) - Decimal(m_min_qty)) % Decimal(m_step_size)
            sugg["Mqty"] = float(Decimal(str(quantity)) - r)
            
    if msg != {}:
        print(msg)
        print(sugg)
    
    return msg, sugg
    

t = validate_order("QNTUSDT", 5.1, "BUY")
print(t)


In [ ]:

# Market DEPTH
def get_order_book(symbol, limit=100):
    params = {
        'symbol': symbol,
        'limit': limit
    }
    return binance_request('/fapi/v1/depth', params)

# VOLUME
def get_ticker(symbol):
    params = {
        'symbol': symbol
    }
    return binance_request('/fapi/v1/ticker/24hr', params)

# Analyse de la liquidite
def analyze_liquidity(symbol):
    order_book = get_order_book(symbol)
    ticker = get_ticker(symbol)
    
    #Spread
    if len(order_book['bids']) > 0 and len(order_book['asks']) > 0:
        best_bid = float(order_book['bids'][0][0])
        best_ask = float(order_book['asks'][0][0])
        spread = (best_ask - best_bid) / best_bid
    else:
        spread = 10
    
    # Volume
    volume = float(ticker['volume'])
    
    #Depth
    bids = pd.DataFrame(order_book['bids'], columns=['price', 'quantity'], dtype=float)
    asks = pd.DataFrame(order_book['asks'], columns=['price', 'quantity'], dtype=float)
    
    total_bids = bids['quantity'].sum()
    total_asks = asks['quantity'].sum()
    
    # Règles de décision
    rules = {
        'spread': spread < 0.005,  # Spread < 0.5%
        'volume': volume > 1000,  # Volume > 1000 unités
        'depth': total_bids > 500 and total_asks > 500  # Profondeur significative
    }
    is_liquid = all(rules.values())
    print(symbol)
    return {
        'symbol': symbol,
        'spread': spread,
        'volume': volume,
        'total_bids': total_bids,
        'total_asks': total_asks,
        'is_liquid': is_liquid,
        'rules': rules
    }
    


In [ ]:
get_order_book("QNTUSDT")

In [ ]:
result = analyze_liquidity(symbol = "QNTUSDT")

# Affichage des résultats
print(f"Symbole: {result['symbol']}")
print(f"Spread Bid-Ask: {result['spread']:.5f}")
print(f"Volume de Trading: {result['volume']}")
print(f"Profondeur des Bids: {result['total_bids']}")
print(f"Profondeur des Asks: {result['total_asks']}")
print(f"Marché Liquide: {'Oui' if result['is_liquid'] else 'Non'}")

# Affichage des règles
for rule, passed in result['rules'].items():
    print(f"Règle '{rule}': {'Passée' if passed else 'Non Passée'}")

In [ ]:
def validation3(symbol, side, order_type, quantity, price, stop_price):
    if order_type == "MARKET":
        price = None
        

In [1]:

def place_order(symbol, side, order_type, quantity, price=None, stop_price=None):
    validation1 = validate_order(symbol, quantity, price)
    
    liquidity = analyze_liquidity(symbol)
    validation2 = liquidity["is_liquid"]
    
    if (validation1 == ({}, {})) and (validation2 is True):
        params = {
            'symbol': symbol,
            'side': side,
            'type': order_type,
            'quantity': quantity
        }
        if order_type == "MARKET":
            return binance_post_request('/fapi/v1/order', params)
        
        if order_type == "LIMIT":
            params['price'] = price
            params['stopPrice'] = stop_price
            
    else:
        print("Validation 1 : ", validation1)
        print("Validation 2 : ", liquidity)
        

place_order(symbol = "QNTUSDT", side="BUY", order_type="MARKET", quantity=2)


NameError: name 'validate_order' is not defined

# Agent : Learning

## Backtest

In [ ]:

from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2022"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

n_SESSION = 100

AGENT_PARAMS = [
    [(1, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(2, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(3, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(4, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(5, "BTC"), CAPITAL, sub_env, n_SESSION]
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("TRIPLEMA", (14, 21, 35), {"floor" : 4})
]

STRATEGIES = [
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4})
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("MOMENTUM", 5, {"floor" : 4}),
    ("MOMENTUM", 6, {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("MOMENTUM", 9, {"floor" : 4})
]

AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    


In [ ]:
AGENT[0].view_report()

In [ ]:
AGENT[0].session.rets_dist.keys()

In [ ]:
AGENT[0].get_report()

In [ ]:
session = AGENT[0].post_event.sessionData
session

In [ ]:
from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2021"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

AGENT_PARAMS = [
    [(1, SYMBOL), CAPITAL, sub_env, n_SESSION]
]


STRATEGIES = [
    ("MOMENTUM", 6, {"floor" : 4})
]


AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    


In [ ]:
AGENT[0].view_report()

In [ ]:
pp = AGENT[0].env.post_event.portfolioData
pp.head()

In [ ]:
dd = AGENT[0].env.post_event.tradeData
dd

In [ ]:
import ast

serie = dd["pnl_pct"].values
serie = dd[dd["state"].apply(lambda x : ast.literal_eval(x)[0]) == 'Close']["pnl_pct"].values

In [ ]:
import ast

dd["state"].apply(lambda x : ast.literal_eval(x)[1]).iloc[-1]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Boxplots
fig_box = go.Figure()
fig_box.add_trace(go.Box(y=serie_pos, name='Positives', marker_color='blue'))
fig_box.add_trace(go.Box(y=serie_neg, name='Négatives', marker_color='red'))
fig_box.update_layout(title_text='Boxplots des valeurs positives et négatives', yaxis_title='Valeur')

fig_box.update_layout(height = 800 , width = 1000,
                          legend = dict(orientation="h",
                                        yanchor="bottom", y=1,
                                        xanchor="right", x=0.5),
                          margin = {'t':0, 'b':0, 'l':10, 'r':0}
                          )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Exemple de série de données
data = serie  # Génère 1000 valeurs aléatoires selon une distribution normale

# Séparer les données en positives et négatives
positive_data = data[data > 0]
negative_data = data[data < 0]

# Afficher les histogrammes
plt.figure(figsize=(12, 6))

# Histogramme des valeurs positives
plt.subplot(1, 2, 1)
plt.hist(positive_data, bins=30, color='blue', alpha=0.7, edgecolor='black')
plt.title('Histogramme des valeurs positives')
plt.xlabel('Valeur')
plt.ylabel('Fréquence')

# Histogramme des valeurs négatives
plt.subplot(1, 2, 2)
plt.hist(negative_data, bins=30, color='red', alpha=0.7, edgecolor='black')
plt.title('Histogramme des valeurs négatives')
plt.xlabel('Valeur')
plt.ylabel('Fréquence')

plt.tight_layout()
plt.show()

# Afficher les KDE
plt.figure(figsize=(12, 6))

# KDE des valeurs positives
sns.kdeplot(positive_data, shade=True, color='blue', label='Positives')
# KDE des valeurs négatives
sns.kdeplot(negative_data, shade=True, color='red', label='Négatives')

plt.title('Estimation de la densité de probabilité')
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = serie

# Séparer les données en positives et négatives
positive_data = data[data > 0]
negative_data = data[data < 0]

# Création de la figure et des sous-graphiques
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

# Histogramme et KDE des valeurs positives
sns.histplot(positive_data, bins=30, kde=True, color='blue', ax=axs[0, 0])
axs[0, 0].set_title('Histogramme et KDE des valeurs positives')
axs[0, 0].set_xlabel('Valeur')
axs[0, 0].set_ylabel('Fréquence/Densité')

# Histogramme et KDE des valeurs négatives
sns.histplot(negative_data, bins=30, kde=True, color='red', ax=axs[0, 1])
axs[0, 1].set_title('Histogramme et KDE des valeurs négatives')
axs[0, 1].set_xlabel('Valeur')
axs[0, 1].set_ylabel('Fréquence/Densité')

# Boxplot des valeurs positives
sns.boxplot(data=positive_data, ax=axs[1, 0], color='blue')
axs[1, 0].set_title('Boxplot des valeurs positives')
axs[1, 0].set_xlabel('Valeur')

# Boxplot des valeurs négatives
sns.boxplot(data=negative_data, ax=axs[1, 1], color='red')
axs[1, 1].set_title('Boxplot des valeurs négatives')
axs[1, 1].set_xlabel('Valeur')

plt.tight_layout()
plt.show()


In [ ]:
port = AGENT[0].env.post_event.portfolioData
port

#### --------------------------------------

# *Multi-Agent* v2(1)

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env, SubEnv

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


TICKERS = ["BTC", "ETH", "SOL", "DASH"]
START = "2022-01-01"
END = "2023-10-01"
INTERVAL = "1d"
CAPITAL = 100

config = {"capital" : CAPITAL, "interval" : INTERVAL,
          "symbols" : TICKERS, "start" : START, "end" : END}
env = Env(**config)

sub_env_1 = SubEnv(symbol="BTC", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_2 = SubEnv(symbol="ETH", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_3 = SubEnv(symbol="SOL", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_4 = SubEnv(symbol="EGLD", capital=CAPITAL, interval=INTERVAL, start=START, end=END)

# Agent details
AGENT_PARAMS = [
    [(1, "BTC"), sub_env_1],
    [(2, "ETH"), sub_env_2],
    [(3, "SOL"), sub_env_3],
    [(4, "EGLD"), sub_env_4]
]

N = len(AGENT_PARAMS)

STRATEGIES = [
    ((1, "BTC"), "MOMENTUM", 3),
    ((2, "ETH"), "TRIPLEMA", (3, 7, 14)),
    ((3, "SOL"), "MOMENTUM", 8),
    ((4, "EGLD"), "TRIPLEMA", (14, 21, 35))
]

In [ ]:
master = MasterAgentThread(env)
master.set_barrier(n=N)

for agent_param, strategie in zip(AGENT_PARAMS, STRATEGIES):
    master.add_agent(*agent_param)
    master.update_params(*strategie)


master.active_agents()
master.run()

In [ ]:
master.agents

In [ ]:
#master.agents[(1, "BTC")].env.get_viz(1, "BTC")
master.agents[(AGENT_PARAMS[0][0])].env.get_viz(*AGENT_PARAMS[0][0])

In [ ]:
master.agents[(AGENT_PARAMS[1][0])].env.get_viz(*AGENT_PARAMS[1][0])

In [ ]:
master.agents[(AGENT_PARAMS[2][0])].env.get_viz(*AGENT_PARAMS[2][0])

In [ ]:
master.agents[(AGENT_PARAMS[3][0])].env.get_viz(*AGENT_PARAMS[3][0])

In [ ]:
master.agents[(4, "EGLD")].env.journal.portfolioData
#master.agents[(3, "SOL")].env.journal.tradesData

In [ ]:
master.agents

In [ ]:
master.agents[(3, "SOL")].env.journal.portfolioData

# *End*

In [ ]:
import threading

threading.enumerate()

# *Multi-Agent v3* Multiprocessing

In [ ]:
import multiprocessing
from multiprocessing import Process, Manager, Queue, Barrier, Event, Semaphore



# Plot

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
class MyGenerator:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.n:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

# Créez une instance de la classe MyGenerator
dix_nombres = MyGenerator(10)

# Utilisez une boucle for pour obtenir les valeurs
for nombre in dix_nombres:
    print(nombre)


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import os 

os.cpu_count()

In [ ]:
import threading

class Master:
    def __init__(self, condition, event):
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print("Maître : J'attends que les agents soient prêts.")
                self.condition.wait()
                print("Maître : Autorisation d'exécuter le travail.")
                self.event.set()
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition, event):
        self.agent_id = agent_id
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print(f"Agent {self.agent_id} : Prêt à travailler.")
                self.condition.notify()
                self.condition.wait()
                self.event.wait()
                self.event.clear()
                print(f"Agent {self.agent_id} : Travail en cours.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez un Event pour autoriser les agents à travailler
event = threading.Event()

# Créez le maître et les agents
master = Master(condition, event)
agents = [Agent(i, condition, event) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
import time
from threading import Thread
from queue import Queue

class Agent:
    def __init__(self, master_queue, name):
        self.master_queue = master_queue
        self.name = name

    def do_task(self, task):
        print(f"Agent {self.name} received task: {task}")
        # Simulating task execution
        time.sleep(2)
        print(f"Agent {self.name} completed task: {task}")
        # Inform the master that the task is completed
        self.master_queue.put((self.name, task))


class Master:
    def __init__(self):
        self.agent_queues = {}

    def add_agent(self, agent):
        agent_queue = Queue()
        self.agent_queues[agent.name] = agent_queue
        Thread(target=self.listen_to_agent, args=(agent, agent_queue)).start()

    def assign_task(self, agent_name, task):
        agent_queue = self.agent_queues.get(agent_name)
        if agent_queue:
            print(f"Master assigned task: {task} to Agent {agent_name}")
            agent_queue.put(task)
        else:
            print(f"Agent {agent_name} not found.")

    def listen_to_agent(self, agent, agent_queue):
        while True:
            task = agent_queue.get()
            if task is None:
                break
            print(f"Master received task completion report from Agent {agent.name} for task: {task}")


# Exemple d'utilisation
if __name__ == "__main__":
    master = Master()

    agents = [Agent(master_queue=master, name=f"Agent{i+1}") for i in range(4)]

    for agent in agents:
        master.add_agent(agent)

    master.assign_task("Agent1", "Task1")
    master.assign_task("Agent2", "Task2")
    master.assign_task("Agent3", "Task3")
    master.assign_task("Agent4", "Task4")

    # Attendez la fin des tâches avant de quitter
    time.sleep(3)  # Simuler le temps nécessaire pour terminer les tâches
    for agent in agents:
        master.add_agent(agent)  # Envoyer un signal de fin à tous les agents


In [ ]:
import multiprocessing
import time
import random

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_queue, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_queue = master_queue
        self.agent_queue = agent_queue

    def run(self):
        while True:
            instruction = self.master_queue.get()

            if instruction == "QUIT":
                break

            # Ajoutez des conditions en fonction de l'instruction
            if instruction == "Commande 1":
                result = f"Agent {self.agent_id} exécute la Commande 1 spécifique."
            elif instruction == "Commande 2":
                result = f"Agent {self.agent_id} exécute la Commande 2 spécifique."
            else:
                result = f"Agent {self.agent_id} exécute une instruction non gérée."

            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_queue = multiprocessing.Queue()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_queue, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            for agent_queue in self.agent_queues:
                self.master_queue.put(instruction)

            time.sleep(1)  # Attendez un certain temps pour simuler le traitement parallèle

            for _ in range(4):
                report = self.master_queue.get()
                print(report)

    def stop_agents(self):
        for _ in range(4):
            self.master_queue.put("QUIT")

        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:
import multiprocessing
import time

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_event, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_event = master_event
        self.agent_queue = agent_queue

    def run(self):
        while True:
            # Wait for the event to be set by the master
            self.master_event.wait()

            instruction = self.agent_queue.get()

            if instruction == "QUIT":
                break

            # Your conditions or logic here
            result = f"Agent {self.agent_id} a exécuté l'instruction: {instruction}"

            # Report the result to the master
            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_event = multiprocessing.Event()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_event, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            # Send the instruction to all agents
            for agent_queue in self.agent_queues:
                agent_queue.put(instruction)

            # Set the event to signal agents to start processing
            self.master_event.set()

            time.sleep(1)  # Simulate parallel processing time

            # Clear the event for the next round
            self.master_event.clear()

            # Collect reports from agents
            for _ in range(4):
                report = self.agent_queues[0].get()  # Assuming all agents provide the same result
                print(report)

    def stop_agents(self):
        # Send the QUIT instruction to all agents
        for agent_queue in self.agent_queues:
            agent_queue.put("QUIT")

        # Wait for agents to finish
        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:


class ProfitManagement:
    def __init__(self, profit_target_percentage, trailing_stop_percentage):
        self.profit_target_percentage = profit_target_percentage
        self.trailing_stop_percentage = trailing_stop_percentage
        self.entry_price = None
        self.profit_target_price = None
        self.trailing_stop_price = None

    def set_entry_price(self, entry_price):
        self.entry_price = entry_price
        self.profit_target_price = entry_price * (1 + self.profit_target_percentage)
        self.trailing_stop_price = entry_price * (1 - self.trailing_stop_percentage)

    def take_profit(self, current_price):
        if current_price >= self.profit_target_price:
            return True
        return False

    def trailing_stop_loss(self, current_price):
        if current_price >= self.trailing_stop_price:
            self.trailing_stop_price = current_price * (1 - self.trailing_stop_percentage)
        return self.trailing_stop_price

# Exemple d'utilisation
profit_target_percentage = 0.05  # Objectif de profit de 5%
trailing_stop_percentage = 0.02  # Trailing stop-loss de 2%

profit_manager = ProfitManagement(profit_target_percentage, trailing_stop_percentage)
profit_manager.set_entry_price(100)  # Prix d'entrée de la position

# Simulation du mouvement des prix
price_history = [100, 110, 120, 115, 125, 130, 135, 130, 140, 145]

for price in price_history:
    if profit_manager.take_profit(price):
        print(f"Prendre profit au prix {price}")
        break
    else:
        trailing_stop_price = profit_manager.trailing_stop_loss(price)
        print(f"Prix actuel : {price}, Trailing stop-loss : {trailing_stop_price}")


In [ ]:
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
from datetime import datetime
from app.binance_api import get_positions, get_history, get_pnl, place_order, get_price, get_all_symbols
from app.db_utils import log_event, get_all_events

def register_callbacks(app):
    @app.callback(
        Output('positions-div', 'children'),
        [Input('interval-component', 'n_intervals')]
    )
    def update_positions(n):
        positions_data = get_positions()
        if 'positions' in positions_data:
            positions = positions_data['positions']
            df_positions = pd.DataFrame(positions)
            df_positions = df_positions[df_positions['positionAmt'].astype(float) != 0]
            return dbc.Table.from_dataframe(df_positions, striped=True, bordered=True, hover=True)
        return "Erreur lors de la récupération des données des positions."

    @app.callback(
        Output('pnl-div', 'children'),
        [Input('interval-component', 'n_intervals')]
    )
    def update_pnl(n):
        pnl_data = get_pnl()
        if 'assets' in pnl_data:
            assets = pnl_data['assets']
            df_pnl = pd.DataFrame(assets)
            df_pnl = df_pnl[df_pnl['walletBalance'].astype(float) > 0]
            return dbc.Table.from_dataframe(df_pnl, striped=True, bordered=True, hover=True)
        return "Erreur lors de la récupération des données du PnL."

    @app.callback(
        [Output('history-div', 'children'), Output('history-graph', 'figure'), Output('detailed-graph', 'figure')],
        [Input('submit-button', 'n_clicks')],
        [State('date-picker-range', 'start_date'), State('date-picker-range', 'end_date')]
    )
    def update_history(n_clicks, start_date, end_date):
        if n_clicks > 0:
            start_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)
            end_timestamp = int(pd.Timestamp(end_date).timestamp() * 1000)
            history_data = get_history(None, start_timestamp, end_timestamp)
            if history_data:
                df_history = pd.DataFrame(history_data)
                table = dbc.Table.from_dataframe(df_history, striped=True, bordered=True, hover=True)

                fig = px.line(df_history, x='time', y='price', color='symbol', title='Prix au Fil du Temps')
                detailed_fig = px.scatter(df_history, x='time', y='price', color='symbol', size='quantity', title='Transactions Individuelles')

                return table, fig, detailed_fig
        return "Sélectionnez une période et cliquez sur 'Afficher l'historique'", {}, {}

    @app.callback(
        Output('assets-div', 'children'),
        [Input('interval-positions-component', 'n_intervals')]
    )
    def update_assets(n):
        positions_data = get_positions()
        if 'positions' in positions_data:
            positions = positions_data['positions']
            df_positions = pd.DataFrame(positions)
            df_positions = df_positions[df_positions['positionAmt'].astype(float) != 0]

            df_positions = df_positions.head(3)

            figures = []
            for asset in df_positions['symbol']:
                start_timestamp = int(pd.Timestamp('today') - pd.DateOffset(days=30).timestamp() * 1000)
                end_timestamp = int(pd.Timestamp('today').timestamp() * 1000)
                history_data = get_history(asset, start_timestamp, end_timestamp)
                if history_data:
                    df_history = pd.DataFrame(history_data)
                    fig = px.line(df_history, x='time', y='price', title=f'Tendances de {asset}')
                    figures.append(dcc.Graph(figure=fig))

            return figures
        return "Erreur lors de la récupération des données des actifs."

    @app.callback(
        Output('order-preview', 'children'),
        [Input('order-symbol', 'value'), Input('order-quantity', 'value'), Input('order-leverage', 'value')]
    )
    def preview_order(symbol, quantity, leverage):
        if symbol and quantity and leverage:
            price_data = get_price(symbol)
            if 'price' in price_data:
                price = float(price_data['price'])
                total = price * quantity * leverage
                return f"Prix actuel : {price} USD, Total : {total} USD (avec un levier de {leverage}x)"
        return ""

    @app.callback(
        [Output('order-result', 'children'), Output('order-result', 'className')],
        [Input('submit-order', 'n_clicks')],
        [State('order-symbol', 'value'),
         State('order-side', 'value'),
         State('order-type', 'value'),
         State('order-quantity', 'value'),
         State('order-leverage', 'value'),
         State('order-price', 'value')]
    )
    def place_order_callback(n_clicks, symbol, side, order_type, quantity, leverage, price):
        if n_clicks > 0:
            if not symbol or not side or not order_type or not quantity or not leverage or (order_type == 'LIMIT' and not price):
                return "Tous les champs sont obligatoires pour les ordres limit", "alert alert-danger"
            if quantity <= 0 or leverage <= 0 or (price and price <= 0):
                return "La quantité, le levier et le prix doivent être des valeurs positives", "alert alert-danger"
            
            result = place_order(symbol, side, order_type, quantity, leverage, price)
            event_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            if 'orderId' in result:
                log_event('ORDER_PLACED', event_time, symbol, side, order_type, quantity, price, 'SUCCESS')
                return f"Ordre placé avec succès : {result}", "alert alert-success"
            else:
                log_event('ORDER_FAILED', event_time, symbol, side, order_type, quantity, price, 'FAILURE')
                return f"Erreur lors du placement de l'ordre : {result}", "alert alert-danger"
        return "", ""

    @app.callback(
        Output('events-div', 'children'),
        [Input('interval-component', 'n_intervals')]
    )
    def display_events(n):
        events = get_all_events()
        df_events = pd.DataFrame(events, columns=['ID', 'Type', 'Time', 'Symbol', 'Side', 'Order Type', 'Quantity', 'Price', 'Result'])
        return dbc.Table.from_dataframe(df_events, striped=True, bordered=True, hover=True)

    @app.callback(
        Output('capital-graph', 'figure'),
        [Input('interval-component', 'n_intervals')]
    )
    def update_capital_graph(n):
        pnl_data = get_pnl()
        if 'assets' in pnl_data:
            assets = pnl_data['assets']
            df_pnl = pd.DataFrame(assets)
            df_pnl = df_pnl[df_pnl['walletBalance'].astype(float) > 0]
            fig = px.line(df_pnl, x='asset', y='walletBalance', title='Évolution du Capital')
            return fig
        return {}

    # Callback pour afficher les points d'entrée et de sortie
    @app.callback(
        Output('history-graph', 'figure'),
        [Input('order-symbol', 'value')]
    )
    def update_entry_exit_points(symbol):
        positions_data = get_positions()
        history_data = get_history(symbol)
        if positions_data and history_data:
            df_positions = pd.DataFrame(positions_data['positions'])
            df_history = pd.DataFrame(history_data)
            fig = px.line(df_history, x='time', y='price', title=f'Historique des prix de {symbol}')
            for index, row in df_positions.iterrows():
                if row['symbol'] == symbol:
                    entry_time = row['entryTime']
                    exit_time = row.get('exitTime')
                    fig.add_scatter(x=[entry_time], y=[row['entryPrice']], mode='markers', name='Point d\'entrée')
                    if exit_time:
                        fig.add_scatter(x=[exit_time], y=[row['exitPrice']], mode='markers', name='Point de sortie')
            return fig
        return px.line(title=f'Aucune donnée disponible pour {symbol}')



In [ ]:
# callbacks.py

import pandas as pd
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from app.binance_api import get_positions, get_pnl, place_order, get_price, get_ohlc_data, get_open_orders
from app.db_utils import log_event, get_all_events

from app.utils import dark_template, processing, ohlc, waterfall

def register_callbacks(app):
    # Callbacks pour mettre à jour les données en temps réel
    @app.callback(
        [Output('waterfall-graph', 'figure'), Output('positions-table', 'children'), Output('risk-indicators', 'children')],
        [Input('interval-component', 'n_intervals'), Input('order-symbol', 'value')]
    )
    def update_wallet_position(n, symbol):
        wallet_data = get_pnl()
        positions_data = get_positions()
        open_orders = get_open_orders()
        
        positions, r_risk, waterfall_fig =  processing(wallet_data, positions_data, open_orders, get_price)
        
        # Tableau des Positions Actuelles
        df_positions = pd.DataFrame(positions)
        positions_table = dbc.Table.from_dataframe(df_positions, striped=True, bordered=True, hover=True)
        
        
        # Indicateurs de suivi et de risque
        risk_indicators = html.Div([
            html.H5("Indicateurs de Suivi et de Risque"),
            html.P(f"Leverage Ratio = {r_risk['leverage_ratio']}%"),
            html.P(f"Maintenance Margin Ratio = {r_risk['maint_margin_ratio']}%"),
            html.P(f"Risk-Reward Ratio = {r_risk['risk_reward_ratio']}%"),
        ])
            
        ohlc_fig = ohlc(positions, symbol, get_ohlc_data)
        
        return waterfall_fig, positions_table, risk_indicators
        
    # Callback pour prévisualiser l'ordre
    @app.callback(
        Output('order-preview', 'children'),
        [Input('order-symbol', 'value'), Input('order-quantity', 'value'), Input('order-leverage', 'value')]
    )
    def preview_order(symbol, quantity, leverage):
        if symbol and quantity and leverage:
            price_data = get_price(symbol)
            if 'price' in price_data:
                price = float(price_data['price'])
                total = price * quantity * leverage
                return f"Prix actuel : {price} USD, Total : {total} USD (avec un levier de {leverage}x)"
        return ""

    # Callback pour afficher ou masquer le champ "Limit Price"
    @app.callback(
        Output('order-price', 'style'),
        [Input('order-type', 'value')]
    )
    def show_hide_limit_price(order_type):
        if order_type == 'LIMIT':
            return {'display': 'block'}
        return {'display': 'none'}

    # Callback pour mettre à jour le cours du symbole sélectionné
    @app.callback(
        Output('current-price', 'children'),
        [Input('order-symbol', 'value')]
    )
    def update_current_price(symbol):
        price_data = get_price(symbol)
        if 'price' in price_data:
            price = float(price_data['price'])
            return f"{price} USD"
        return "Erreur lors de la récupération du prix"

    # Callback pour afficher le graphique OHLC
    @app.callback(
        Output('ohlc-graph', 'figure'),
        [Input('order-symbol', 'value')]
    )
    def update_ohlc_graph(symbol):
        end_date = datetime.now()
        start_date = end_date - timedelta(days=7)
        interval = "1h"
        
        ohlc_data = get_ohlc_data(symbol, start_date, end_date, interval)
        
        if ohlc_data:
            df_ohlc = pd.DataFrame(ohlc_data)
            fig = go.Figure(data=[go.Candlestick(
                x=df_ohlc['time'],
                open=df_ohlc['open'],
                high=df_ohlc['high'],
                low=df_ohlc['low'],
                close=df_ohlc['close']
            )])
            y_range = [min(df_ohlc["low"]) * 0.99, max(df_ohlc["high"]) * 1.01]
            fig = fig.update_yaxes(range=y_range, showgrid=False)
            
            max_time = df_ohlc['time'].max() + pd.Timedelta(hours=8)
            fig.update_xaxes(range=[df_ohlc['time'].min(), max_time])
            
            fig.update_xaxes(rangeslider_visible=False, showgrid=False,
                             rangeselector=dict(
                                    buttons=list([
                                        dict(count=30, label="30min", step="minute", stepmode="backward"),
                                        dict(count=4, label="4h", step="hour", stepmode="backward"),
                                        dict(count=12, label="12h", step="hour", stepmode="backward"),
                                        dict(count=1, label="1d", step="day", stepmode="backward"),
                                        dict(count=7, label="7w", step="day", stepmode="backward")
                                    ]),
                                    font=dict(color="blue")
                                )
                             )
            
            fig.update_layout(
                height = 600,
                legend = dict(orientation="h",
                                    yanchor="bottom", y=1,
                                    xanchor="right", x=0.5
                                    ),
                          margin = {'t':0, 'b':0, 'l':10, 'r':0},
                          template=dark_template, showlegend=False
                          )           
            return fig
        return {}

    # Callback pour placer un ordre Long / Short
    @app.callback(
        [Output('order-result', 'children'), Output('order-result', 'className')],
        [Input('order-long', 'n_clicks'), Input('order-short', 'n_clicks')],
        [State('order-symbol', 'value'),
         State('order-type', 'value'),
         State('order-quantity', 'value'),
         State('order-leverage', 'value'),
         State('order-price', 'value')]
    )
    def place_order_callback(n_clicks_long, n_clicks_short, symbol, order_type, quantity, leverage, price):
        if n_clicks_long > 0:
            if not symbol or not order_type or not quantity or not leverage or (order_type == 'LIMIT' and not price):
                return "Tous les champs sont obligatoires pour les ordres limit", "alert alert-danger"
            if quantity <= 0 or leverage <= 0 or (price and price <= 0):
                return "La quantité, le levier et le prix doivent être des valeurs positives", "alert alert-danger"
            
            side = "BUY"
            result = place_order(symbol, side, order_type, quantity, leverage, price)
            event_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            if 'orderId' in result:
                log_event('ORDER_PLACED', event_time, symbol, side, order_type, quantity, price, 'SUCCESS')
                return f"Ordre placé avec succès : {result}", "alert alert-success"
            else:
                log_event('ORDER_FAILED', event_time, symbol, side, order_type, quantity, price, 'FAILURE')
                return f"Erreur lors du placement de l'ordre : {result}", "alert alert-danger"
        
        if n_clicks_short > 0:
            if not symbol or not order_type or not quantity or not leverage or (order_type == 'LIMIT' and not price):
                return "Tous les champs sont obligatoires pour les ordres limit", "alert alert-danger"
            if quantity <= 0 or leverage <= 0 or (price and price <= 0):
                return "La quantité, le levier et le prix doivent être des valeurs positives", "alert alert-danger"
            
            side = "SELL"
            result = place_order(symbol, side, order_type, quantity, leverage, price)
            event_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            if 'orderId' in result:
                log_event('ORDER_PLACED', event_time, symbol, side, order_type, quantity, price, 'SUCCESS')
                return f"Ordre placé avec succès : {result}", "alert alert-success"
            else:
                log_event('ORDER_FAILED', event_time, symbol, side, order_type, quantity, price, 'FAILURE')
                return f"Erreur lors du placement de l'ordre : {result}", "alert alert-danger"
        return "", ""

    # Callback pour afficher les événements
    @app.callback(
        Output('events-div', 'children'),
        [Input('interval-component', 'n_intervals')]
    )
    def display_events(n):
        events = get_all_events()
        df_events = pd.DataFrame(events, columns=['ID', 'Type', 'Time', 'Symbol', 'Side', 'Order Type', 'Quantity', 'Price', 'Result'])
        return dbc.Table.from_dataframe(df_events, striped=True, bordered=True, hover=True)


# Tableau de Bord de Trading Binance Futures

Cette application est un tableau de bord de trading pour Binance Futures, offrant diverses fonctionnalités pour gérer et suivre vos positions de trading.

## Fonctionnalités Principales

### Visualisation des Positions Actuelles
- Affiche les positions ouvertes en temps réel.
- Met à jour les données des positions toutes les 10 secondes.

### Statistiques du PnL
- Affiche les statistiques du profit et perte (PnL) en temps réel.
- Met à jour les données du PnL toutes les 10 secondes.

### Historique des Positions
- Permet de sélectionner une période spécifique pour afficher l'historique des ordres.
- Affiche un tableau des ordres passés et un graphique des prix au fil du temps pour les symboles sélectionnés.
- Affiche des points d'entrée et de sortie sur le graphique des prix si des positions existent pour le symbole sélectionné.

### Graphiques Détaillés
- Visualise les transactions individuelles avec des points d'entrée et de sortie sur les graphiques de prix.
- Affiche un graphique détaillé des transactions pour les symboles sélectionnés.

### Actifs Détenus
- Affiche les actifs actuellement détenus, limités à trois actifs au maximum.
- Met à jour les graphiques des actifs détenus toutes les 10 secondes.

### Placement d'Ordres
- Permet de placer des ordres de type Market et Limit.
- Sélection du symbole de trading parmi toutes les paires disponibles sur Binance.
- Sélection du côté de l'ordre (Long/Short) et du type d'ordre (Market/Limit).
- Prévisualisation du total de l'ordre avant de le placer.
- Affichage du résultat de l'ordre après le placement (succès ou échec).

### Journal des Événements
- Affiche un journal des événements de trading, y compris les ordres placés, réussis ou échoués.
- Met à jour les événements toutes les 10 secondes.

### Évolution du Capital
- Affiche un graphique de l'évolution du capital en temps réel.
- Met à jour les données du capital toutes les 10 secondes.

## Technologies Utilisées
- **Dash** : Pour la visualisation des données et les composants interactifs.
- **Dash Bootstrap Components** : Pour le style et les composants visuels.
- **API Binance** : Pour récupérer les données de trading.

## Fonctionnalités Interactives
- **Sélection de la Période** : Permet de sélectionner une période spécifique pour afficher l'historique des positions.
- **Sélection de la Paire de Trading** : Permet de sélectionner une paire de trading parmi toutes les paires disponibles sur Binance.
- **Points d'Entrée et de Sortie** : Affiche les points d'entrée et de sortie sur les graphiques de prix pour les positions existantes.
- **Placement d'Ordres** : Permet de placer des ordres de trading avec prévisualisation du total de l'ordre.
- **Graphiques Dynamiques** : Affiche des graphiques détaillés des transactions et de l'évolution du capital.

## Mise à Jour en Temps Réel
- **Positions Actuelles** : Mise à jour toutes les 10 secondes.
- **Statistiques du PnL** : Mise à jour toutes les 10 secondes.
- **Actifs Détenus** : Mise à jour toutes les 10 secondes.
- **Journal des Événements** : Mise à jour toutes les 10 secondes.
- **Évolution du Capital** : Mise à jour toutes les 10 secondes.
